# DeepQL with topics no trainable

In [1]:
# Optimize the use of GPUs
# https://datascience.stackexchange.com/questions/23895/multi-gpu-in-keras
# https://keras.io/getting-started/faq/#how-can-i-run-a-keras-model-on-multiple-gpus
# https://stackoverflow.com/questions/56316451/how-to-use-specific-gpus-in-keras-for-multi-gpu-training

In [2]:
# import tensorflow as tf
import keras
# from __future__ import print_function, division
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
from annoy import AnnoyIndex
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
from keras.layers import Conv1D, Input, Add, Activation, Dropout, Embedding, MaxPooling1D, \
    GlobalMaxPool1D, Flatten, Dense, Concatenate, BatchNormalization
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.initializers import TruncatedNormal
from keras import optimizers

from methods.baseline import Baseline
from methods.experiments import Experiment
from methods.evaluation import Evaluation
from methods.retrieval import Retrieval

import os
from keras_bert import load_vocabulary
import random

from keras.initializers import RandomUniform, RandomNormal, Ones

from keras_bert import load_trained_model_from_checkpoint
from keras_bert import compile_model, get_model
from keras.layers import GlobalAveragePooling1D

## required for semi-hard triplet loss:
import tensorflow as tf
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import dtypes
from sklearn.utils.extmath import softmax

from keras.layers import concatenate, Add, Lambda, merge, Average, Maximum
from keras.optimizers import Adam, Nadam
import _pickle as pickle

from keras.layers import Layer
from keras import backend as K
    
# %matplotlib inline

Using TensorFlow backend.


## Auxiliary methods

In [3]:
%env epochs 1000
%env base openoffice

env: epochs=1000
env: base=openoffice


## Configurações Globais

In [4]:
MAX_SEQUENCE_LENGTH_T = 100 # 100
'''
    Bert length title and description
    # Eclipse
    # 200   recall@25 = 
    # 100   recall@25 = 
    # 20    recall@25 = 0.53
    # Netbeans
    # 200   recall@25 = 
    # 100   recall@25 = 
    # 20    recall@25 = 0.64
    # Open office
    # 200   recall@25 = 
    # 100   recall@25 = 0.60
    # 20    recall@25 = 0.54
'''
MAX_SEQUENCE_LENGTH_D = 100 # 500 

'''
    Bert length description
    # Eclipse
    # 200   recall@25 = 0.58
    # 100   recall@25 = 0.57
    # 20    recall@25 = 0.53
    # Netbeans
    # 200   recall@25 = 
    # 100   recall@25 = 0.66
    # 20    recall@25 = 0.64
    # Open office
    # 200   recall@25 = 0.58
    # 100   recall@25 = 0.58
    # 30-40 recall@25 = 0.56
    # 20    recall@25 = 0.54
'''

TOPIC_SEQUENCE = 30 # Number of topics considered
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000
'''
    Configuration
'''
epochs = int(os.environ['epochs'])
freeze_train = .1 # 10% with freeze weights
best_loss = 1
best_epoch = 0
verbose = 0
loss = 1

### Parse bugs preproprecessed

In [5]:
# Domain to use
DOMAIN = os.environ['base']
METHOD = 'deepQL_topics_{}'.format(epochs)
PREPROCESSING = 'bert'
TOKEN = 'bert'
# Dataset paths
DIR = 'data/processed/{}/{}'.format(DOMAIN, PREPROCESSING)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Glove embeddings
GLOVE_DIR='data/embed'
# Save model
SAVE_PATH = '{}_preprocessing_{}_feature@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_preprocessing_{}_feature_@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

In [6]:
DIR

'data/processed/openoffice/bert'

In [7]:
print("*********")
print("{} for {} epochs in {}".format(METHOD, epochs, DOMAIN))
print("*********")

*********
deepQL_topics_1000 for 1000 epochs in openoffice
*********


In [8]:
pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
model_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [9]:
token_dict = load_vocabulary(vocab_path)

In [10]:
"Total vocabulary: {}".format(len(token_dict))

'Total vocabulary: 30522'

In [11]:
baseline = Baseline(DOMAIN, DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D,
                   token_dict['[CLS]'], token_dict['[SEP]'])
evaluation = Evaluation(verbose=0)
retrieval = Retrieval()
experiment = Experiment(baseline, evaluation)

In [12]:
experiment.set_retrieval(retrieval, baseline, DOMAIN)

In [13]:
baseline.info_dict

{'bug_severity': 6,
 'bug_status': 3,
 'component': 144,
 'priority': 5,
 'product': 41,
 'version': 539}

#### Loading bug ids in memory

In [14]:
experiment.load_ids()
len(baseline.bug_ids)

Reading bug ids


98070

#### Dicionário de títulos e descrições

In [15]:
%%time

experiment.load_bugs(TOKEN)
len(baseline.sentence_dict)


CPU times: user 23.3 s, sys: 1.83 s, total: 25.1 s
Wall time: 24.5 s


#### Hashing bugs by buckets

In [16]:
issues_by_buckets = experiment.get_buckets_for_bugs()

#### Prepare the train and test

In [17]:
%%time

experiment.prepare_dataset(issues_by_buckets, path_train='train_chronological', path_test='test_chronological')
# Read and create the test queries duplicates
retrieval.create_queries()

CPU times: user 35.3 ms, sys: 336 µs, total: 35.7 ms
Wall time: 32.5 ms


In [18]:
baseline.train_data[:10]

[[7499, 14267],
 [7499, 14451],
 [7499, 14335],
 [14267, 14451],
 [14267, 14335],
 [14451, 14335],
 [109857, 109858],
 [55321, 45346],
 [55321, 25797],
 [55321, 39359]]

#### Recovery bug ids from train

In [19]:
bug_train_ids = experiment.get_train_ids(baseline.train_data)

### Export the corpus train

In [20]:
if EXTRACT_CORPUS:
    corpus = []
    export_file = open(os.path.join(DIR, 'corpus_train.txt'), 'w')
    for bug_id in tqdm(baseline.bug_set):
        bug = baseline.bug_set[bug_id]
        title = bug['title']
        desc = bug['description']
        export_file.write("{}\n{}\n".format(title, desc))
    export_file.close()

In [21]:
idx = np.random.choice(baseline.bug_ids, 1)[0]
baseline.bug_set[idx]

{'bug_severity': '2\n',
 'bug_status': '1\n',
 'component': '89\n',
 'creation_ts': '2004-09-19 02:45:00 +0000',
 'delta_ts': '2013-08-07 14:41:36 +0000',
 'description': '[CLS] happens with writer and cal ##c steps to reproduce . ( assumes you have word and excel files are associated with o ##oo ) - open windows explorer and find a word or excel document - double - click on it to open the file in o ##oo - click on file > properties crash ! : ( this does not happen if you double - click on an o ##oo file type in windows explorer to open the file in o ##oo . kevin [SEP]',
 'description_segment': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

In [22]:
"Train ", len(baseline.dup_sets_train)

('Train ', 5502)

### Train ids

In [23]:
# data - path
# batch_size - 128
# n_neg - 1
def batch_iterator(self, retrieval, model, data, dup_sets, bug_ids, 
                   batch_size, n_neg, issues_by_buckets, TRIPLET_HARD=False, FLOATING_PADDING=False):
    # global train_data
    # global self.dup_sets
    # global self.bug_ids
    # global self.bug_set

    random.shuffle(data)

    batch_features = {'title' : [], 'desc' : [], 'info' : [], 'topics' : []}
    n_train = len(data)

    batch_triplets, batch_bugs_anchor, batch_bugs_pos, batch_bugs_neg, batch_bugs = [], [], [], [], []

    all_bugs = bug_ids #list(issues_by_buckets.keys())
    buckets = retrieval.buckets

    for offset in range(batch_size):
        anchor, pos = data[offset][0], data[offset][1]
        batch_bugs_anchor.append(anchor)
        batch_bugs_pos.append(pos)
        batch_bugs.append(anchor)
        batch_bugs.append(pos)
        #batch_bugs += dup_sets[anchor]

    for anchor, pos in zip(batch_bugs_anchor, batch_bugs_pos):
        while True:
            neg = self.get_neg_bug(anchor, buckets[issues_by_buckets[anchor]], issues_by_buckets, all_bugs)
            bug_anchor = self.bug_set[anchor]
            bug_pos = self.bug_set[pos]
            if neg not in self.bug_set:
                continue
            batch_bugs.append(neg)
            batch_bugs_neg.append(neg)
            bug_neg = self.bug_set[neg]
            break
        
        # triplet bug and master
        batch_triplets.append([anchor, pos, neg])
    
    random.shuffle(batch_bugs)
    title_ids = np.full((len(batch_bugs), MAX_SEQUENCE_LENGTH_T), 0)
    description_ids = np.full((len(batch_bugs), MAX_SEQUENCE_LENGTH_D), 0)
    for i, bug_id in enumerate(batch_bugs):
        bug = self.bug_set[bug_id]
        self.read_batch_bugs(batch_features, bug, index=i, title_ids=title_ids, description_ids=description_ids)

    batch_features['title'] = { 'token' : np.array(batch_features['title']), 'segment' : title_ids }
    batch_features['desc'] = { 'token' : np.array(batch_features['desc']), 'segment' : description_ids }
    batch_features['info'] = np.array(batch_features['info'])
    batch_features['topics'] = np.array(batch_features['topics'])
    
    sim = np.asarray([issues_by_buckets[bug_id] for bug_id in batch_bugs])

    input_sample = {}

    input_sample = { 'title' : batch_features['title'], 
                        'description' : batch_features['desc'], 
                            'info' : batch_features['info'],
                            'topics' : batch_features['topics'] }

    return batch_triplets, input_sample, sim #sim

In [24]:
# %%time

batch_size = 64
batch_size_test = 128

bug_test_ids = experiment.get_test_ids(baseline.test_data)

# we want a constant validation group to have a frame of reference for model performance
batch_triplets_valid, valid_input_sample, valid_sim = batch_iterator(baseline, retrieval, None, 
                                                                                      baseline.test_data, 
                                                                                      baseline.dup_sets_test,
                                                                                      bug_test_ids,
                                                                                      batch_size_test, 1,
                                                                                      issues_by_buckets)

validation_sample = [valid_input_sample['title']['token'], valid_input_sample['title']['segment'], 
                   valid_input_sample['description']['token'], valid_input_sample['description']['segment'],
                   valid_input_sample['info'], valid_input_sample['topics'], valid_sim]

# Categorical columns
number_of_columns_info = valid_input_sample['info'].shape[1]
# Max sequence title
MAX_SEQUENCE_LENGTH_T = valid_input_sample['title']['token'].shape[1]
MAX_SEQUENCE_LENGTH_D = valid_input_sample['description']['token'].shape[1]

In [25]:
valid_input_sample['title']['token'].shape, \
valid_input_sample['description']['token'].shape, \
valid_input_sample['title']['segment'].shape, \
valid_input_sample['description']['segment'].shape, \
valid_input_sample['info'].shape, valid_input_sample['topics'].shape, \
valid_sim.shape

((384, 100), (384, 100), (384, 100), (384, 100), (384, 738), (384, 30), (384,))

### Validar entrada

In [26]:
# %%time 

#baseline.display_batch(baseline.train_data, baseline.dup_sets_train, bug_train_ids, 5)

In [27]:
"Test ", len(baseline.test_data)

('Test ', 4116)

## Experiment

## Propose

https://github.com/tqtg/DuplicateBugFinder

### BERT

https://github.com/CyberZHG/keras-bert

In [28]:
def bert_model(MAX_SEQUENCE_LENGTH, name):
    layer_num = 8
#     model = load_trained_model_from_checkpoint(
#             config_path,
#             model_path,
#             training=True,
#             trainable=True,
#             seq_len=MAX_SEQUENCE_LENGTH,
#     )
    model = load_trained_model_from_checkpoint(
        config_path,
        model_path,
        training=True,
        use_adapter=True,
        seq_len=MAX_SEQUENCE_LENGTH,
        trainable=['Encoder-{}-MultiHeadSelfAttention-Adapter'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-FeedForward-Adapter'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-MultiHeadSelfAttention-Norm'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-FeedForward-Norm'.format(i + 1) for i in range(layer_num)],
    )
#     model = get_model(
#         token_num=len(token_dict),
#         head_num=10,
#         transformer_num=layer_num,
#         embed_dim=100,
#         feed_forward_dim=100,
#         seq_len=MAX_SEQUENCE_LENGTH,
#         pos_num=MAX_SEQUENCE_LENGTH,
#         dropout_rate=0.05,
#     )
    compile_model(model)
    inputs = model.inputs[:2]
    layers = ['Encoder-{}-MultiHeadSelfAttention-Adapter', 'Encoder-{}-FeedForward-Adapter', 
     'Encoder-{}-MultiHeadSelfAttention-Norm', 'Encoder-{}-FeedForward-Norm']
    outputs = []
    for i in range(1, 13):
        outputs += [ model.get_layer(layer.format(layer_num)).output for layer in layers ]
    outputs = Average()(outputs)
    #outputs = model.get_layer('Extract').output
    outputs = GlobalAveragePooling1D()(outputs)
    outputs = Dense(300, activation='tanh')(outputs)
    
    model = Model(inputs, outputs, name='FeatureBERTGenerationModel{}'.format(name))
    
    return model

### MLP

In [29]:
def mlp_model(input_size):
    info_input = Input(shape=(input_size, ), name='Feature_BugInput')
    input_size = 300
    
    for units in [64, 32]:
        layer = Dense(units, activation='tanh', kernel_initializer='random_uniform')(info_input)
    
    layer = Dense(input_size, activation='tanh')(info_input)
    
    mlp_feature_model = Model(inputs=[info_input], outputs=[layer], name = 'FeatureMlpGenerationModel')
    
    return mlp_feature_model

### Single MLP

In [30]:
def topic_model(input_size):
    info_input = Input(shape=(input_size, ), name='FeatureTopic_BugInput')
    input_size = 300
    
    layer = Dense(input_size, activation='tanh')(info_input)
    
    mlp_feature_model = Model(inputs=[info_input], outputs=[layer], name = 'FeatureTopicMlpGenerationModel')
    
    return mlp_feature_model

### Siamese model

In [31]:
def pairwise_distance(feature, squared=False):
    """Computes the pairwise distance matrix with numerical stability.

    output[i, j] = || feature[i, :] - feature[j, :] ||_2

    Args:
      feature: 2-D Tensor of size [number of data, feature dimension].
      squared: Boolean, whether or not to square the pairwise distances.

    Returns:
      pairwise_distances: 2-D Tensor of size [number of data, number of data].
    """
    pairwise_distances_squared = math_ops.add(
        math_ops.reduce_sum(math_ops.square(feature), axis=[1], keepdims=True),
        math_ops.reduce_sum(
            math_ops.square(array_ops.transpose(feature)),
            axis=[0],
            keepdims=True)) - 2.0 * math_ops.matmul(feature,
                                                    array_ops.transpose(feature))

    # Deal with numerical inaccuracies. Set small negatives to zero.
    pairwise_distances_squared = math_ops.maximum(pairwise_distances_squared, 0.0)
    # Get the mask where the zero distances are at.
    error_mask = math_ops.less_equal(pairwise_distances_squared, 0.0)

    # Optionally take the sqrt.
    if squared:
        pairwise_distances = pairwise_distances_squared
    else:
        pairwise_distances = math_ops.sqrt(
            pairwise_distances_squared + math_ops.to_float(error_mask) * 1e-16)

    # Undo conditionally adding 1e-16.
    pairwise_distances = math_ops.multiply(
        pairwise_distances, math_ops.to_float(math_ops.logical_not(error_mask)))

    num_data = array_ops.shape(feature)[0]
    # Explicitly set diagonals to zero.
    mask_offdiagonals = array_ops.ones_like(pairwise_distances) - array_ops.diag(
        array_ops.ones([num_data]))
    pairwise_distances = math_ops.multiply(pairwise_distances, mask_offdiagonals)
    return pairwise_distances

def masked_maximum(data, mask, dim=1):
    """Computes the axis wise maximum over chosen elements.

    Args:
      data: 2-D float `Tensor` of size [n, m].
      mask: 2-D Boolean `Tensor` of size [n, m].
      dim: The dimension over which to compute the maximum.

    Returns:
      masked_maximums: N-D `Tensor`.
        The maximized dimension is of size 1 after the operation.
    """
    axis_minimums = math_ops.reduce_min(data, dim, keepdims=True)
    masked_maximums = math_ops.reduce_max(
        math_ops.multiply(data - axis_minimums, mask), dim,
        keepdims=True) + axis_minimums
    return masked_maximums

def masked_minimum(data, mask, dim=1):
    """Computes the axis wise minimum over chosen elements.

    Args:
      data: 2-D float `Tensor` of size [n, m].
      mask: 2-D Boolean `Tensor` of size [n, m].
      dim: The dimension over which to compute the minimum.

    Returns:
      masked_minimums: N-D `Tensor`.
        The minimized dimension is of size 1 after the operation.
    """
    axis_maximums = math_ops.reduce_max(data, dim, keepdims=True)
    masked_minimums = math_ops.reduce_min(
        math_ops.multiply(data - axis_maximums, mask), dim,
        keepdims=True) + axis_maximums
    return masked_minimums

In [32]:
def triplet_loss(vects):
    margin = 1.
    labels = vects[:, :1]
 
    labels = tf.cast(labels, dtype='int32')

    embeddings = tf.cast(vects[:, 1:], dtype='float32')

    ### Code from Tensorflow function [tf.contrib.losses.metric_learning.triplet_semihard_loss] starts here:
    
    # Reshape [batch_size] label tensor to a [batch_size, 1] label tensor.
    # lshape=array_ops.shape(labels)
    # assert lshape.shape == 1
    # labels = array_ops.reshape(labels, [lshape[0], 1])

    # Build pairwise squared distance matrix.
    pdist_matrix = pairwise_distance(embeddings, squared=True)
    # Build pairwise binary adjacency matrix.
    adjacency = math_ops.equal(labels, array_ops.transpose(labels))
    # Invert so we can select negatives only.
    adjacency_not = math_ops.logical_not(adjacency)

    # global batch_size  
    batch_size = array_ops.size(labels) # was 'array_ops.size(labels)'

    # Compute the mask.
    pdist_matrix_tile = array_ops.tile(pdist_matrix, [batch_size, 1])
    mask = math_ops.logical_and(
        array_ops.tile(adjacency_not, [batch_size, 1]),
        math_ops.greater(
            pdist_matrix_tile, array_ops.reshape(
                array_ops.transpose(pdist_matrix), [-1, 1])))
    mask_final = array_ops.reshape(
        math_ops.greater(
            math_ops.reduce_sum(
                math_ops.cast(mask, dtype=dtypes.float32), 1, keepdims=True),
            0.0), [batch_size, batch_size])
    mask_final = array_ops.transpose(mask_final)

    adjacency_not = math_ops.cast(adjacency_not, dtype=dtypes.float32)
    mask = math_ops.cast(mask, dtype=dtypes.float32)

    # negatives_outside: smallest D_an where D_an > D_ap.
    negatives_outside = array_ops.reshape(
        masked_minimum(pdist_matrix_tile, mask), [batch_size, batch_size])
    negatives_outside = array_ops.transpose(negatives_outside)

    # negatives_inside: largest D_an.
    negatives_inside = array_ops.tile(
        masked_maximum(pdist_matrix, adjacency_not), [1, batch_size])
    semi_hard_negatives = array_ops.where(
        mask_final, negatives_outside, negatives_inside)

    loss_mat = math_ops.add(margin, pdist_matrix - semi_hard_negatives)

    mask_positives = math_ops.cast(
        adjacency, dtype=dtypes.float32) - array_ops.diag(
        array_ops.ones([batch_size]))

    # In lifted-struct, the authors multiply 0.5 for upper triangular
    #   in semihard, they take all positive pairs except the diagonal.
    num_positives = math_ops.reduce_sum(mask_positives)

    semi_hard_triplet_loss_distance = math_ops.truediv(
        math_ops.reduce_sum(
            math_ops.maximum(
                math_ops.multiply(loss_mat, mask_positives), 0.0)),
        num_positives,
        name='triplet_semihard_loss')
    
    ### Code from Tensorflow function semi-hard triplet loss ENDS here.
    return semi_hard_triplet_loss_distance

def quintet_loss(inputs):
    margin = 1.
    labels = inputs[:, :1]
 
    labels = tf.cast(labels, dtype='int32')

    embeddings = inputs[:, 1:]

    # Build pairwise squared distance matrix.
    pdist_matrix = pairwise_distance(embeddings, squared=True)
    # Build pairwise binary adjacency matrix.
    adjacency = math_ops.equal(labels, array_ops.transpose(labels))
    # Invert so we can select negatives only.
    adjacency_not = math_ops.logical_not(adjacency)

    # global batch_size  
    batch_size = array_ops.size(labels) # was 'array_ops.size(labels)'

    # Compute the mask.
    pdist_matrix_tile = array_ops.tile(pdist_matrix, [batch_size, 1])
    mask = math_ops.logical_and(
        array_ops.tile(adjacency_not, [batch_size, 1]),
        math_ops.greater(
            pdist_matrix_tile, array_ops.reshape(
                array_ops.transpose(pdist_matrix), [-1, 1])))
    mask_final = array_ops.reshape(
        math_ops.greater(
            math_ops.reduce_sum(
                math_ops.cast(mask, dtype=dtypes.float32), 1, keepdims=True),
            0.0), [batch_size, batch_size])
    
    mask_final = array_ops.transpose(mask_final)

    adjacency_not = math_ops.cast(adjacency_not, dtype=dtypes.float32)
    mask = math_ops.cast(mask, dtype=dtypes.float32)

    # negatives_outside: smallest D_an where D_an > D_ap.
    negatives_outside = array_ops.reshape(
        masked_minimum(pdist_matrix_tile, mask), [batch_size, batch_size])
    negatives_outside = array_ops.transpose(negatives_outside)

    # negatives_inside: largest D_an.
    negatives_inside = array_ops.tile(
        masked_maximum(pdist_matrix, adjacency_not), [1, batch_size])

    semi_hard_negatives = array_ops.where(
        mask_final, negatives_outside, negatives_inside)
    
    semi_hard_negatives_mask = math_ops.equal(pdist_matrix, masked_maximum(pdist_matrix, adjacency_not))
    
    # Remove false negatives with similarity equal to the true negatives
    semi_hard_negatives_mask = tf.reshape(tf.cast(semi_hard_negatives_mask, 'int32'), (-1, 1)) * tf.reshape(tf.cast(adjacency_not, 'int32'), (-1, 1))
    semi_hard_negatives_mask = tf.cast(tf.reshape(semi_hard_negatives_mask, (batch_size, batch_size)), 'bool')
    
    # Recovery the bug label from semi-hard-negatives
    label_matrix = tf.repeat(tf.reshape(labels, (1, -1)), repeats=[batch_size], axis=0)
    semi_hard_negatives_ids = tf.reshape(label_matrix, (-1, 1)) * tf.cast(tf.reshape(semi_hard_negatives_mask, (-1, 1)), 'int32')
    semi_hard_negatives_ids = tf.reshape(semi_hard_negatives_ids, (batch_size, batch_size))

    i = tf.constant(0)
    most_freq_matrix = tf.Variable([])
    def most_frequent(i, most_freq_matrix):
        batch = tf.gather(semi_hard_negatives_ids, i)
        neg_label_default = [tf.unique(batch)[0][0]]
        batch = tf.boolean_mask(batch, tf.greater(batch, 0))
        unique, _, count = tf.unique_with_counts(batch)
        max_occurrences = tf.reduce_max(count)
        max_cond = tf.equal(count, max_occurrences)
        max_numbers = tf.squeeze(tf.gather(unique, tf.where(max_cond)))
        max_numbers = tf.cond(tf.cast(tf.size(unique) > 1, tf.bool), lambda: unique[0], lambda: max_numbers)
        max_numbers = tf.cond(tf.cast(tf.shape(unique) == 0, tf.bool), 
                              lambda: neg_label_default, 
                              lambda: max_numbers)
        most_freq_matrix = tf.concat([most_freq_matrix, [max_numbers]], axis=0)
        return [tf.add(i, 1), most_freq_matrix]
    _, negatives_ids = tf.while_loop(lambda i, _: i<batch_size, 
                                        most_frequent, 
                                        [i, most_freq_matrix],
                                       shape_invariants=[i.get_shape(),
                                                   tf.TensorShape([None])])
    negatives_ids = tf.cast(negatives_ids, 'int32')
    labels_neg = tf.reshape(negatives_ids, (-1, 1))
    mask_negatives = math_ops.equal(labels_neg, semi_hard_negatives_ids)
    mask_negatives = tf.cast(mask_negatives, 'float32')
    labels_neg = tf.cast(labels_neg, 'float32')
    
    mask_positives = math_ops.cast(
        adjacency, dtype=dtypes.float32) - array_ops.diag(
        array_ops.ones([batch_size]))
    
    # In lifted-struct, the authors multiply 0.5 for upper triangular
    #   in semihard, they take all positive pairs except the diagonal.
    num_positives = math_ops.reduce_sum(mask_positives)

    # Include the anchor to positives
    mask_positives_centroids = math_ops.cast(adjacency, dtype=dtypes.float32)

    # centroid pos 
    embed_pos = tf.matmul(mask_positives, embeddings)
    num_of_pos = tf.reduce_sum(mask_positives_centroids, axis=1, keepdims=True)
    centroid_embed_pos = tf.math.xdivy(embed_pos, num_of_pos)
    labels_pos = tf.cast(labels, dtype=dtypes.float32)

    # centroid negs
    embed_neg = tf.matmul(mask_negatives, embeddings)
    num_of_neg = tf.reduce_sum(mask_negatives, axis=1, keepdims=True)
    centroid_embed_neg = tf.math.xdivy(embed_neg, num_of_neg)
    
    i = tf.constant(0)
    batch_centroid_matrix = tf.Variable([])
    def iter_centroids(i, batch_centroid_matrix):
        # anchor
        anchor = [tf.gather(embeddings, i)]
        label_pos = [tf.gather(labels_pos, i)]
        # centroid pos
        centroid_pos = [tf.gather(centroid_embed_pos, i)]
        # centroid neg
        centroid_neg = [tf.gather(centroid_embed_neg, i)]
        label_neg = [tf.gather(labels_neg, i)]
        # new batch
        new_batch = tf.concat([anchor, centroid_pos, centroid_neg], axis=0)
        new_labels = tf.concat([label_pos, label_pos, label_neg], axis=0)
        # Batch anchor + centroid_positive + centroid_negative
        batch_anchor_centroids = tf.concat([new_labels, new_batch], axis=1)
        TL_single_triplet = triplet_loss(batch_anchor_centroids)
        batch_centroid_matrix = tf.concat([batch_centroid_matrix, [TL_single_triplet]], axis=0)
        
        return [tf.add(i, 1), batch_centroid_matrix]
    _, batch_centroid_matrix = tf.while_loop(lambda i, a: i<batch_size, 
                                        iter_centroids, 
                                        [i, batch_centroid_matrix],
                                       shape_invariants=[i.get_shape(),
                                                   tf.TensorShape([None])])
    
    TL_centroid = tf.reduce_mean(batch_centroid_matrix)
    TL = triplet_loss(inputs)
    TL_pos = tf.constant(0.0)
    TL_neg = tf.constant(0.0) #tf.reduce_mean(batch_centroid_matrix_neg)
   
    return K.stack([TL, TL_pos, TL_neg, TL_centroid], axis=0)

def quintet_trainable(inputs):
    TL = inputs[0]
    TL_pos = inputs[1]
    TL_neg = inputs[2]
    TL_centroid = inputs[3]
    TL_anchor_w = inputs[4]
    TL_pos_w = inputs[5]
    TL_neg_w = inputs[6]
    TL_centroid_w = inputs[7]
    
    TL_anchor_w = math_ops.maximum(TL_anchor_w, 0.0)
    TL_neg_w = math_ops.maximum(TL_neg_w, 0.0)
    TL_centroid_w = math_ops.maximum(TL_centroid_w, 0.0)

    sum_of_median = tf.reduce_sum([ TL * TL_anchor_w, TL_centroid * TL_centroid_w]) # , TL_neg * TL_neg_w,
    sum_of_weigths = tf.reduce_sum([TL_anchor_w, TL_centroid_w])
    weigthed_median = tf.truediv(sum_of_median, sum_of_weigths)    
    return K.stack([weigthed_median, TL_anchor_w, TL_pos_w, TL_neg_w, TL_centroid_w, TL, TL_pos, TL_neg, TL_centroid], axis=0)

def custom_loss(y_true, y_pred):
    return tf.reduce_mean(y_pred[0])

def TL_w_anchor(y_true, y_pred):
    return tf.reduce_mean(y_pred[1])
def TL_w_pos(y_true, y_pred):
    return tf.reduce_mean(y_pred[2])
def TL_w_neg(y_true, y_pred):
    return tf.reduce_mean(y_pred[3])
def TL_w_centroid(y_true, y_pred):
    return tf.reduce_mean(y_pred[4])
def TL(y_true, y_pred):
    return tf.reduce_mean(y_pred[5])
def TL_pos(y_true, y_pred):
    return tf.reduce_mean(y_pred[6])
def TL_neg(y_true, y_pred):
    return tf.reduce_mean(y_pred[7])
def TL_centroid(y_true, y_pred):
    return tf.reduce_mean(y_pred[8])

### Propose

In [33]:

def siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, topic_feature_model, sequence_length_info, 
                  sequence_length_t, sequence_length_d, sequence_length_topics, name):
  
    # Title
    bug_t_token = Input(shape = (sequence_length_t, ), name = 'title_token_{}'.format(name))
    bug_t_segment = Input(shape = (sequence_length_t, ), name = 'title_segment_{}'.format(name))
    # Description
    bug_d_token = Input(shape = (sequence_length_d, ), name = 'desc_token_{}'.format(name))
    bug_d_segment = Input(shape = (sequence_length_d, ), name = 'desc_segment_{}'.format(name))
    # Categorical
    bug_i = Input(shape = (sequence_length_info, ), name = 'info_{}'.format(name))
    # Topics
    bug_topics = Input(shape = (sequence_length_topics, ), name='topics_{}'.format(name))
    
    bug_t_feat = title_feature_model([bug_t_token, bug_t_segment])
    bug_d_feat = desc_feature_model([bug_d_token, bug_d_segment])
    bug_i_feat = categorical_feature_model(bug_i)
    bug_topics_feat = topic_feature_model(bug_topics)
    
    #bug_feature_output = Add(name = 'merge_features_{}'.format(name))([bug_i_feat, bug_t_feat, bug_d_feat])
    bug_feature_output = concatenate([bug_i_feat, bug_t_feat, bug_d_feat, bug_topics_feat], name = 'merge_features_{}'.format(name))
    
    bug_feature_model = Model(inputs=[bug_t_token, bug_t_segment, bug_d_token, bug_d_segment, bug_i, bug_topics], outputs=[bug_feature_output], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

In [34]:
class QuintetWeights(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(QuintetWeights, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = tf.reshape(self.add_weight(name='quintet_kernel_weight', 
                                      shape=(input_shape[0], self.output_dim),
                                      initializer=keras.initializers.Ones(),
#                                       initializer=keras.initializers.RandomUniform(minval=0.0, maxval=1.0, seed=None),
                                      trainable=False), (1, 1))
        super(QuintetWeights, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        x = tf.reshape(x, (1, 1))
        return [K.dot(x, self.kernel), self.kernel]

    def compute_output_shape(self, input_shape):
        return [input_shape, input_shape]
    
def max_margin_objective(encoded_anchor, decay_lr=1):
    
    input_labels = Input(shape=(1,), name='input_label')    # input layer for labels
    inputs = np.concatenate([encoded_anchor.input, [input_labels]], -1).tolist()
    
    encoded_anchor = encoded_anchor.output
    
    feature = concatenate([input_labels, encoded_anchor])  # concatenating the labels + embeddings
    
    TL_loss = Lambda(quintet_loss, name='quintet_loss')(feature)
    
    tl_l = Lambda(lambda x:tf.reshape(x[0], (1,)), name='TL')(TL_loss)
    tl_l_p = Lambda(lambda x:tf.reshape(x[1], (1,)), name='TL_pos')(TL_loss)
    tl_l_n = Lambda(lambda x:tf.reshape(x[2], (1,)), name='TL_neg')(TL_loss)
    tl_l_c = Lambda(lambda x:tf.reshape(x[3], (1, )), name='TL_centroid')(TL_loss)
    
    TL_w = QuintetWeights(output_dim=1)(tl_l)
    TL_pos_w = QuintetWeights(output_dim=1)(tl_l_p)
    TL_neg_w = QuintetWeights(output_dim=1)(tl_l_n)
    TL_centroid_w = QuintetWeights(output_dim=1)(tl_l_c)
    
    TL_weight = Lambda(lambda x:tf.reshape(x[1], (1,)), name='TL_weight')(TL_w)
    TL_pos_weight = Lambda(lambda x:tf.reshape(x[1], (1,)), name='TL_pos_weight')(TL_pos_w)
    TL_neg_weight = Lambda(lambda x:tf.reshape(x[1], (1,)), name='TL_neg_weight')(TL_neg_w)
    TL_centroid_weight = Lambda(lambda x:tf.reshape(x[1], (1,)), name='TL_centroid_weight')(TL_centroid_w)
    
    output = concatenate([tl_l, tl_l_p, tl_l_n, tl_l_c, TL_weight, TL_pos_weight, TL_neg_weight, TL_centroid_weight])
    output = Lambda(quintet_trainable, name='quintet_trainable')(output)
    
    similarity_model = Model(inputs = inputs, outputs = output, name = 'Similarity_Model')

    # setup the optimization process 
    similarity_model.compile(optimizer='adam', loss=custom_loss, metrics=[TL_w_anchor, TL_w_pos, TL_w_neg, TL_w_centroid,
                                                                         TL, TL_pos, TL_neg, TL_centroid]) 
    # metrics=[pos_distance, neg_distance, custom_margin_loss]

    return similarity_model

In [35]:
def save_loss(result):
    with open(os.path.join(DIR,'{}_log.pkl'.format(METHOD)), 'wb') as f:
        pickle.dump(result, f)
    print("=> result saved!")

In [36]:
# Domain to use
result = { 'train' : [], 'test' : [] }
limit_train = int(epochs * freeze_train) # 10% de 1000 , 100 epocas
METHOD = 'deepQL_topics_{}'.format(limit_train)
SAVE_PATH = '{}_preprocessing_{}_feature@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_preprocessing_{}_feature_@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)

In [37]:
%%time

# Inspired on https://'pastebin.com/TaGFdcBA
# TODO: https://stackoverflow.com/questions/49941903/keras-compute-cosine-distance-between-two-flattened-outputs
keras.backend.clear_session()

# Clear GPU memory
# from numba import cuda
# cuda.select_device(0)
# cuda.close()

# Feature models
'''
    cnn_model
    lstm_model
    mlp_model
'''
title_feature_model = bert_model(MAX_SEQUENCE_LENGTH_T, 'Title')
desc_feature_model = bert_model(MAX_SEQUENCE_LENGTH_D, 'Description')
categorical_feature_model = mlp_model(number_of_columns_info)
topic_feature_model = topic_model(TOPIC_SEQUENCE)

# Similarity model
encoded_anchor = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, topic_feature_model,
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, TOPIC_SEQUENCE, 'in')

similarity_model = max_margin_objective(encoded_anchor, decay_lr=1)

# cnn_feature_model.summary()
# lstm_feature_model.summary()
similarity_model.summary()

'''
    Experiment
'''
print("Total of ", limit_train)
for epoch in range(limit_train):
    batch_triplet_train, \
        train_input_sample, train_sim = batch_iterator(baseline, retrieval, encoded_anchor, baseline.train_data, 
                                                       baseline.dup_sets_train, bug_train_ids, 
                                                           batch_size, 1, issues_by_buckets, TRIPLET_HARD=False)
    train_batch = [train_input_sample['title']['token'], train_input_sample['title']['segment'], 
                   train_input_sample['description']['token'], train_input_sample['description']['segment'],
                   train_input_sample['info'], train_input_sample['topics'], train_sim]
    
#     if epoch == 10:
#         similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, decay_lr=0.1)
    
    h = similarity_model.train_on_batch(x=train_batch, y=train_sim)
    h_validation = similarity_model.test_on_batch(x=validation_sample, y=valid_sim)
    
    # save results
    result['train'].append(h)
    result['test'].append(h_validation)
    
    if( (epoch+1) % 10 == 0 or (epoch+1 == limit_train) ):
        save_loss(result)
    
    if (epoch+1 == limit_train): #(epoch > 1 and epoch % 10 == 0) or (epoch+1 == epochs):
        recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, verbose, encoded_anchor, issues_by_buckets, 
                                                               bug_train_ids, method='bert-topic')
        print(("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}\n" +
               "TL_w: {:.2f}, TL_pos_w: {:.2f}, TL_neg_w: {:.2f}, TL_centroid_w: {:.2f}\n" + 
                "TL: {:.2f}, TL_pos: {:.2f}, TL_neg: {:.2f}, TL_centroid: {:.2f}, " +
              "recall@25: {:.2f}").format(epoch+1, h[0], h_validation[0], h[1], h[2], h[3], 
                                          h[4], h[5], h[6], h[7], h[8], recall))
    else:
        print(("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}\n" +
               "TL_w: {:.2f}, TL_pos_w: {:.2f}, TL_neg_w: {:.2f}, TL_centroid_w: {:.2f}\n" + 
              "TL: {:.2f}, TL_pos: {:.2f}, TL_neg: {:.2f}, TL_centroid: {:.2f}").format(
            epoch+1, h[0], h_validation[0], h[1], h[2], h[3], h[4], h[5], h[6], h[7], h[8]))
    loss = h[0]
    
    if loss < best_loss:
        best_loss = loss
        best_epoch = epoch+1

# experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
# experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
print('Best_epoch={}, Best_loss={:.2f}, Recall@25={:.2f}'.format(best_epoch, best_loss, recall))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 738)          0                                            
__________________________________________________________________________________________________
title_token_in (InputLayer)     (None, 100)          0                                            
__________________________________________________________________________________________________
title_segment_in (InputLayer)   (None, 100)          0                                            
__________________________________

/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "




Epoch: 1 Loss: 1.06, Loss_test: 0.78
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.70, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 1.41
Epoch: 2 Loss: 0.60, Loss_test: 0.45
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.73, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.46
Epoch: 3 Loss: 0.42, Loss_test: 0.41
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.75, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.08
Epoch: 4 Loss: 0.40, Loss_test: 0.40
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.77, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.03
Epoch: 5 Loss: 0.37, Loss_test: 0.39
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.73, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00
Epoch: 6 Loss: 0.36, Loss_test: 0.38
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.70, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.02
Epoch: 7 Loss: 0.42, Loss_test: 0.37
TL_w: 1.00, TL_pos_

Epoch: 51 Loss: 0.14, Loss_test: 0.28
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.17, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.11
Epoch: 52 Loss: 0.14, Loss_test: 0.28
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.22, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.05
Epoch: 53 Loss: 0.15, Loss_test: 0.28
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.26, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.05
Epoch: 54 Loss: 0.06, Loss_test: 0.28
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.11, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.01
Epoch: 55 Loss: 0.12, Loss_test: 0.29
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.18, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.06
Epoch: 56 Loss: 0.09, Loss_test: 0.29
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.17, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.02
Epoch: 57 Loss: 0.13, Loss_test: 0.29
TL_w: 1.00, TL

In [38]:
recall

0.804

In [39]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

'data/processed/openoffice/bert/exported_rank_deepQL_topics_100.txt'

In [40]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [41]:
experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(limit_train)))
experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(limit_train)), verbose=1)

Saved model 'modelos/model_bert_preprocessing_deepQL_topics_100_feature_100epochs_64batch(openoffice).h5' to disk


In [42]:
len(result['train']), len(result['test'])

(100, 100)

In [43]:
model = similarity_model.get_layer('concatenate_2')
output = Lambda(quintet_trainable, name='quintet_trainable')(model.output)
inputs = similarity_model.inputs
model = Model(inputs = inputs, outputs = output, name = 'Similarity_Model')

# setup the optimization process 
model.compile(optimizer='adam', loss=custom_loss, metrics=[TL_w_anchor, TL_w_pos, TL_w_neg, TL_w_centroid,
                                                                     TL, TL_pos, TL_neg, TL_centroid])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 738)          0                                            
__________________________________________________________________________________________________
title_token_in (InputLayer)     (None, 100)          0                                            
__________________________________________________________________________________________________
title_segment_in (InputLayer)   (None, 100)          0                                            
__________________________________________________________________________________________________
desc_token_in (InputLayer)      (None, 100)          0                                            
__________________________________________________________________________________________________
desc_segme

In [44]:
# Domain to use
METHOD = 'deepQL_topics_{}'.format(epochs)
SAVE_PATH = '{}_preprocessing_{}_feature@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_preprocessing_{}_feature_@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)

In [45]:
end_train = epochs - limit_train
for epoch in range(limit_train, end_train):
    batch_triplet_train, \
        train_input_sample, train_sim = batch_iterator(baseline, retrieval, model, baseline.train_data, 
                                                       baseline.dup_sets_train, bug_train_ids, 
                                                           batch_size, 1, issues_by_buckets, TRIPLET_HARD=False)
    train_batch = [train_input_sample['title']['token'], train_input_sample['title']['segment'], 
                   train_input_sample['description']['token'], train_input_sample['description']['segment'],
                   train_input_sample['info'], train_input_sample['topics'], train_sim]
    

    h = model.train_on_batch(x=train_batch, y=train_sim)
    h_validation = model.test_on_batch(x=validation_sample, y=valid_sim)
    
    # save results
    result['train'].append(h)
    result['test'].append(h_validation)
    
    if( (epoch+1) % 10 == 0 or (epoch+1 == end_train )):
        save_loss(result)
    
    print(("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}\n" +
               "TL_w: {:.2f}, TL_pos_w: {:.2f}, TL_neg_w: {:.2f}, TL_centroid_w: {:.2f}\n" + 
              "TL: {:.2f}, TL_pos: {:.2f}, TL_neg: {:.2f}, TL_centroid: {:.2f}").format(
            epoch+1, h[0], h_validation[0], h[1], h[2], h[3], h[4], h[5], h[6], h[7], h[8]))

/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch: 101 Loss: 0.08, Loss_test: 0.23
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.13, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.04
Epoch: 102 Loss: 0.05, Loss_test: 0.24
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.09, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.01
Epoch: 103 Loss: 0.11, Loss_test: 0.24
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.08, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.14
Epoch: 104 Loss: 0.05, Loss_test: 0.24
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.11, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00
Epoch: 105 Loss: 0.05, Loss_test: 0.26
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.09, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00
Epoch: 106 Loss: 0.07, Loss_test: 0.25
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.10, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.05
Epoch: 107 Loss: 0.09, Loss_test: 0.25
TL_w: 1

Epoch: 152 Loss: 0.08, Loss_test: 0.20
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.13, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.03
Epoch: 153 Loss: 0.06, Loss_test: 0.20
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.11, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.01
Epoch: 154 Loss: 0.03, Loss_test: 0.20
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.05, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.01
Epoch: 155 Loss: 0.12, Loss_test: 0.20
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.19, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.06
Epoch: 156 Loss: 0.05, Loss_test: 0.19
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.07, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.03
Epoch: 157 Loss: 0.08, Loss_test: 0.19
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.13, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.03
Epoch: 158 Loss: 0.05, Loss_test: 0.20
TL_w: 1

Epoch: 203 Loss: 0.08, Loss_test: 0.22
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.13, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.02
Epoch: 204 Loss: 0.08, Loss_test: 0.23
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.09, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.08
Epoch: 205 Loss: 0.06, Loss_test: 0.24
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.05, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.06
Epoch: 206 Loss: 0.04, Loss_test: 0.23
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.07, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.01
Epoch: 207 Loss: 0.04, Loss_test: 0.23
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.08, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00
Epoch: 208 Loss: 0.03, Loss_test: 0.23
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.05, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.01
Epoch: 209 Loss: 0.04, Loss_test: 0.23
TL_w: 1

Epoch: 254 Loss: 0.03, Loss_test: 0.22
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.03, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.03
Epoch: 255 Loss: 0.11, Loss_test: 0.22
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.16, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.07
Epoch: 256 Loss: 0.03, Loss_test: 0.22
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.06, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00
Epoch: 257 Loss: 0.07, Loss_test: 0.22
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.09, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.06
Epoch: 258 Loss: 0.06, Loss_test: 0.21
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.12, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00
Epoch: 259 Loss: 0.04, Loss_test: 0.21
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.04, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.04
=> result saved!
Epoch: 260 Loss: 0.05, Loss_t

...

Epoch: 662 Loss: 0.04, Loss_test: 0.23
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.05, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.03
Epoch: 663 Loss: 0.03, Loss_test: 0.24
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.04, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.01
Epoch: 664 Loss: 0.03, Loss_test: 0.23
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.03, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.02
Epoch: 665 Loss: 0.02, Loss_test: 0.24
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.04, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00
Epoch: 666 Loss: 0.03, Loss_test: 0.24
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.03, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.02
Epoch: 667 Loss: 0.04, Loss_test: 0.23
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.05, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.04
Epoch: 668 Loss: 0.03, Loss_test: 0.24
TL_w: 1

Epoch: 713 Loss: 0.04, Loss_test: 0.18
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.04, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.03
Epoch: 714 Loss: 0.02, Loss_test: 0.18
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.04, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00
Epoch: 715 Loss: 0.04, Loss_test: 0.18
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.05, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.04
Epoch: 716 Loss: 0.04, Loss_test: 0.19
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.05, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.03
Epoch: 717 Loss: 0.06, Loss_test: 0.19
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.09, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.02
Epoch: 718 Loss: 0.01, Loss_test: 0.19
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.02, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00
Epoch: 719 Loss: 0.08, Loss_test: 0.19
TL_w: 1

Epoch: 764 Loss: 0.01, Loss_test: 0.22
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.02, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00
Epoch: 765 Loss: 0.01, Loss_test: 0.20
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.03, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00
Epoch: 766 Loss: 0.02, Loss_test: 0.20
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.03, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00
Epoch: 767 Loss: 0.03, Loss_test: 0.20
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.06, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00
Epoch: 768 Loss: 0.04, Loss_test: 0.23
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.08, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00
Epoch: 769 Loss: 0.02, Loss_test: 0.23
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.03
=> result saved!
Epoch: 770 Loss: 0.02, Loss_t

Epoch: 815 Loss: 0.03, Loss_test: 0.18
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.04, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.01
Epoch: 816 Loss: 0.02, Loss_test: 0.18
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.04, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00
Epoch: 817 Loss: 0.02, Loss_test: 0.19
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.03, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00
Epoch: 818 Loss: 0.00, Loss_test: 0.19
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.00, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00
Epoch: 819 Loss: 0.02, Loss_test: 0.20
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.04, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.01
=> result saved!
Epoch: 820 Loss: 0.02, Loss_test: 0.20
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.03, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.01
Epoch: 821 Loss: 0.02, Loss_t

Epoch: 866 Loss: 0.01, Loss_test: 0.19
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.02, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00
Epoch: 867 Loss: 0.01, Loss_test: 0.20
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.02, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00
Epoch: 868 Loss: 0.03, Loss_test: 0.20
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.06, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00
Epoch: 869 Loss: 0.03, Loss_test: 0.21
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.06, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00
=> result saved!
Epoch: 870 Loss: 0.01, Loss_test: 0.21
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.02, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00
Epoch: 871 Loss: 0.01, Loss_test: 0.21
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.01, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00
Epoch: 872 Loss: 0.04, Loss_t

In [46]:
len(result['train']), len(result['test'])

(900, 900)

In [47]:
encoded = model.get_layer('merge_features_in')
output = encoded.output
inputs = similarity_model.inputs[:-1]
encoded_anchor = Model(inputs = inputs, outputs = output, name = 'Similarity_Model')

In [48]:
SAVE_PATH.replace('@number_of_epochs@', str(epochs))

'bert_preprocessing_deepQL_topics_1000_feature1000epochs_64batch(openoffice)'

In [49]:
experiment.save_model(model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
"Model saved"

Saved model 'modelos/model_bert_preprocessing_deepQL_topics_1000_feature_1000epochs_64batch(openoffice).h5' to disk


'Model saved'

In [50]:
recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, 1, encoded_anchor, issues_by_buckets, 
                                                               bug_train_ids, method='bert-topic')
print(("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}\n" +
       "TL_w: {:.2f}, TL_pos_w: {:.2f}, TL_neg_w: {:.2f}, TL_centroid_w: {:.2f}\n" + 
        "TL: {:.2f}, TL_pos: {:.2f}, TL_neg: {:.2f}, TL_centroid: {:.2f}, " +
      "recall@25: {:.2f}").format(epoch+1, h[0], h_validation[0], h[1], h[2], h[3], h[4], h[5], h[6], h[7], h[8], recall))


Epoch: 900 Loss: 0.02, Loss_test: 0.24
TL_w: 1.00, TL_pos_w: 1.00, TL_neg_w: 1.00, TL_centroid_w: 1.00
TL: 0.05, TL_pos: 0.00, TL_neg: 0.00, TL_centroid: 0.00, recall@25: 0.85


In [51]:
recall

0.848

In [52]:
exported_rank[:20]

['49152:49146|62680:0.323016881942749,53192:0.30926841497421265,38810:0.30442893505096436,70165:0.29995566606521606,52761:0.2893027663230896,97139:0.2873495817184448,37707:0.28443384170532227,51035:0.28400570154190063,38182:0.27973151206970215,69983:0.277793824672699,22723:0.27328479290008545,79446:0.2716836929321289,73301:0.26608890295028687,84728:0.26602429151535034,94008:0.2607090473175049,62793:0.25822168588638306,22184:0.25738924741744995,88567:0.25545060634613037,111617:0.25527071952819824,3587:0.25435030460357666,30133:0.25293952226638794,86193:0.2525026202201843,62531:0.2515033483505249,9723:0.25100868940353394,39003:0.25080031156539917,103472:0.2506982684135437,2162:0.2494732141494751,56109:0.24906522035598755,40907:0.24575716257095337',
 '73735:73736|73736:1.0,37026:0.4814743995666504,45989:0.4125303626060486,39868:0.4111284613609314,62242:0.4064904451370239,68992:0.38627713918685913,77431:0.3774927258491516,43556:0.3774667978286743,44794:0.37673866748809814,38512:0.375336527

In [53]:
print("Total of queries:", len(retrieval.test))

Total of queries: 4116


In [54]:
print(SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)))

bert_preprocessing_deepQL_topics_1000_feature_1000epochs_64batch(openoffice)


In [55]:
encoded_anchor.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 738)          0                                            
__________________________________________________________________________________________________
title_token_in (InputLayer)     (None, 100)          0                                            
__________________________________________________________________________________________________
title_segment_in (InputLayer)   (None, 100)          0                                            
__________________________________________________________________________________________________
desc_token_in (InputLayer)      (None, 100)          0                                            
__________________________________________________________________________________________________
desc_segme

In [56]:
print(len(exported_rank))

1922


In [57]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
print(EXPORT_RANK_PATH)

data/processed/openoffice/bert/exported_rank_deepQL_topics_1000.txt


In [58]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [59]:
report = experiment.evaluation.evaluate(EXPORT_RANK_PATH)
print(report)

{'0 - recall_at_1': 0.588, '3 - recall_at_15': 0.811, '4 - recall_at_20': 0.832, '1 - recall_at_5': 0.735, '5 - recall_at_25': 0.848, '2 - recall_at_10': 0.79}
